In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.facecolor'] = 'w'
import random
from matplotlib.animation import FuncAnimation
from IPython import display
from scipy.spatial.distance import pdist, squareform

In [2]:
from time import time
def runtime(t0):
    dt = time() - t0
    if dt < 60:
        message = f'{dt:.0f} sec.'
    elif dt < 3600:
        message = f'{dt/60:.0f} min.'
    else:
        h = int(dt//3600)
        m = int((dt%3600) / 60)
        message = f'{h} h. {m} min.'
    print(message)

In [6]:
"""
all modified from:
author: Jake Vanderplas
email: vanderplas@astro.washington.edu
website: http://jakevdp.github.com
license: BSD
Please feel free to use and modify this, but keep the above information. Thanks!
"""
class Particle_System():
    def __init__(self, initial_state=[[10, 10, 10, 10],
                                      [10, 10, -10, 10],
                                      [10, 10, 10, -10]], 
                 particle_size=1,
                 particle_mass=1,
                 bounds = [-2, 2, -2, 2],
                 box_size=100, color='cornflowerblue'):
        
        self.box_size = box_size
        self.bounds = bounds
        self.color = color
        self.particle_size = particle_size
        self.system_state = np.asarray(init_state, dtype=float)
        self.M = particle_mass * np.ones(self.system_state.shape[0])
        
        
    def update_state(self, dt):
        
        # update positions
        self.system_state[:, :2] += dt * self.system_state[:, 2:]

        # find pairs of particles undergoing a collision
        distance = squareform(pdist(self.system_state[:, :2]))
        distance_th = 2 * self.particle_size
        idx1, idx2 = np.where(distance < distance_th)
        unique = (idx1 < idx2)
        idx1 = idx1[unique]
        idx2 = idx2[unique]

        # update velocities of colliding pairs
        for i1, i2 in zip(idx1, idx2):
            # mass
            m1 = self.M[i1]
            m2 = self.M[i2]

            # location vector
            r1 = self.system_state[i1, :2]
            r2 = self.system_state[i2, :2]

            # velocity vector
            v1 = self.system_state[i1, 2:]
            v2 = self.system_state[i2, 2:]

            # relative location & velocity vectors
            r_rel = r1 - r2
            v_rel = v1 - v2

            # momentum vector of the center of mass
            v_cm = (m1 * v1 + m2 * v2) / (m1 + m2)

            # collisions of spheres reflect v_rel over r_rel
            rr_rel = np.dot(r_rel, r_rel)
            vr_rel = np.dot(v_rel, r_rel)
            v_rel = 2 * r_rel * vr_rel / rr_rel - v_rel

            # assign new velocities
            self.system_state[i1, 2:] = v_cm + v_rel * m2 / (m1 + m2)
            self.system_state[i2, 2:] = v_cm - v_rel * m1 / (m1 + m2) 
        #------------------------
        # if beyond box boundaries: elastic bounce against walls
        out_w = (self.system_state[:, 0] < self.bounds[0] + self.particle_size)
        out_e = (self.system_state[:, 0] > self.bounds[1] - self.particle_size)
        out_n = (self.system_state[:, 1] < self.bounds[2] + self.particle_size)
        out_s = (self.system_state[:, 1] > self.bounds[3] - self.particle_size)

        self.system_state[out_w, 0] = self.bounds[0] + self.particle_size
        self.system_state[out_e, 0] = self.bounds[1] - self.particle_size
        self.system_state[out_n, 1] = self.bounds[2] + self.particle_size
        self.system_state[out_s, 1] = self.bounds[3] - self.particle_size
        # bounce
        self.system_state[out_w | out_e, 2] *= -1
        self.system_state[out_n | out_s, 3] *= -1
        #------------------------

In [15]:
#------------------------------------------------------------
# set up initial state
t0 = time()
num_particles = 30
num_frames    = 500
np.random.seed(0)
init_state = np.random.random((num_particles, 4))
# init_state[:, :2] *= 3.9

all_particles = Particle_System(init_state, particle_size=0.05)
dt = 1. / 30 # 30fps


#------------------------------------------------------------
# set up figure and animation
fig = plt.figure()
fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
ax = fig.add_subplot(111, aspect='equal', autoscale_on=False,
                     xlim=(-3.2, 3.2), ylim=(-2.4, 2.4))

# particles holds the locations of the particles
particles, = ax.plot([], [], color='cornflowerblue', marker='o', ms=6)

# rect is the box edge
rect = plt.Rectangle(all_particles.bounds[::2],
                     all_particles.bounds[1] - all_particles.bounds[0],
                     all_particles.bounds[3] - all_particles.bounds[2],
                     ec='none', lw=2, fc='none')
ax.add_patch(rect)
time_text = ax.text(0.3*all_particles.bounds[0], 1.05*all_particles.bounds[3], '')


def init():
    """initialize animation"""
    global all_particles, rect, time_text
    particles.set_data([], [])
    rect.set_edgecolor('none')
    time_text.set_text('')
    return particles, rect, time_text

def animate(frame):
    """perform animation step"""
    global all_particles, rect, dt, ax, fig, time_text
    all_particles.update_state(dt)

    ms = int(fig.dpi * 2 * all_particles.particle_size * fig.get_figwidth()
             / np.diff(ax.get_xbound())[0])
    
    # update pieces of the animation
    rect.set_edgecolor('k')
    particles.set_data(all_particles.system_state[:, 0], all_particles.system_state[:, 1])
    particles.set_markersize(ms)
    
    if frame % 20 == 0:
        time_text.set_text(f'frameme = {frame}/{num_frames}')
    return particles, rect, time_text

ani = FuncAnimation(fig, animate, frames=num_frames,
                    interval=10, blit=True, init_func=init)
video = ani.to_html5_video()
html = display.HTML(video)
display.display(html)
plt.close()
runtime(t0)

21 sec.


In [4]:
# ================= URANIUM ==================================
density = 10 # nuclei per unit
num_nuclei = density**2
nuclei_size = 100
nuclei = np.zeros(num_nuclei, dtype=[('position', float, 2),
                                     ('color',    float, 4)])
color_nuclei    = np.repeat([[0.949, 0.894, 0.227, 1]], repeats=num_nuclei, axis=0)
grid_values = np.linspace(0, 1, density)
idx = 0
uranium_state = []
for x in grid_values:
    for y in grid_values:
        nuclei['position'][idx] = x, y
        uranium_state.append(x, y, 0, 0)
        idx += 1

ax_nuclei = ax.scatter(nuclei['position'][:, 0], nuclei['position'][:, 1], 
                       s=nuclei_size, edgecolors='k', facecolors=nuclei['color'])

TypeError: append() takes exactly one argument (4 given)

In [10]:
t0 = time()
fig, ax = plt.subplots()
fig.set_size_inches(6, 6)
lines = ax.plot([], 'o')
line = lines[0]
# Construct the scatter that will update during animation
# scat = ax.scatter([], [], edgecolors='b',
#                   facecolors='b')

# other static stuff
ax.set_aspect('equal')
box_size = 200
ax.set_xlim(0, box_size)
ax.set_ylim(0, box_size)
time_text = ax.text(40, box_size+2, '')

num_particles = 25
vmax = .0
particles = []
for _ in range(num_particles):
    x0  = box_size*random.uniform(0, 1)
    y0  = box_size*random.uniform(0, 1)
    vx0 = vmax*random.uniform(-1, 1)
    vy0 = vmax*random.uniform(-1, 1)
    initial_state1 = [x0, y0, vx0, vy0]
    p = Particle(initial_state=initial_state1, 
                 box_size=box_size)
    particles.append(p)
    

v0 = 1
p0 = Particle(initial_state=[102, 25, v0, v0], 
              box_size=box_size, color='r')
particles.append(p0)

def init():
    """initialize animation"""
    line.set_data([], [])
#     scat.set_offsets()
    time_text.set_text('')
        
    return line, time_text

def animate(frame):
    x, y = [], []
    for p in particles:
        p.move(1)
        xy = p.get_position()
        x.append(xy[0])
        y.append(xy[1])
    line.set_data((x, y))
    line.set_color(color='limegreen')
    
    if frame % 10 == 0:
        time_text.set_text(f'time = {frame:.0f}')
    return line, time_text

num_frames = 500
    
animt = FuncAnimation(fig, animate, init_func=init, frames=num_frames, interval=30, repeat=True)
video = animt.to_html5_video()
html = display.HTML(video)
display.display(html)
plt.close()
runtime(t0)

31 sec.


In [5]:
animt.save('animt.gif', writer='imagemagick')

In [ ]:
def move(position, velocity, dt, box_size=1):
    for idx, (x, v) in enumerate(zip(position, velocity)):
        if (x[0] < 0 or x[0] > box_size):
            # particle at the x boundary
            v[0] = - v[0]

        if (x[1] < 0 or x[1] > box_size):
            # particle at the y boundary
            v[1] = - v[1]

        x += v * dt
        position[idx], velocity[idx]
    return position, velocity

In [130]:
def move(position, velocity, dt, box_size=1):
    for idx, (x, v) in enumerate(zip(position, velocity)):
        if (x[0] < 0 or x[0] > box_size):
            # particle at the x boundary
            v[0] = - v[0]

        if (x[1] < 0 or x[1] > box_size):
            # particle at the y boundary
            v[1] = - v[1]

        x += v * dt
        position[idx], velocity[idx] = x, v
#     if (position[0] < 0 or position[0] > box_size):
#         # particle at the x boundary
#         velocity[0] = - velocity[0]

#     if (position[1] < 0 or position[1] > box_size):
#         # particle at the y boundary
#         velocity[1] = - velocity[1]

#     position += velocity * dt
    return position, velocity

In [118]:
discs = np.zeros(3, dtype=[('position', float, 2),
                                   ('velocity', float, 2),
                                   ('color',    float, 3)])
vv = np.random.uniform(-1, 1, (3, 2))

In [131]:
discs

array([([0.79607361, 0.9953603 ], [-0.00880484, -0.00299458], [0., 0., 0.]),
       ([0.84730769, 0.41445653], [ 0.00839476,  0.00921533], [0., 0., 0.]),
       ([0.12749888, 0.84064091], [ 0.00281129,  0.00377297], [0., 0., 0.])],
      dtype=[('position', '<f8', (2,)), ('velocity', '<f8', (2,)), ('color', '<f8', (3,))])

In [162]:
discs['color']

array([[0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],


In [205]:
t0 = time()
# fig, ax = plt.subplots()
# fig.set_size_inches(6, 6)
fig = plt.figure(figsize=(6, 6))
# box_size = 
ax = fig.add_axes([0, 0, 1, 1], frameon=True)
ax.set_xlim(0, 1), ax.set_xticks([])
ax.set_ylim(0, 1), ax.set_yticks([])

num_discs = 500
disc_size = 60
vmax = 1e-3
discs = np.zeros(num_discs, dtype=[('position', float, 2),
                                   ('velocity', float, 2),
                                   ('color',    float, 4)])

np.random.seed(123)
discs['position'] = np.random.uniform(0, 1, (num_discs, 2))
discs['velocity'] = np.random.uniform(-vmax, vmax, (num_discs, 2))
discs['color']    = np.repeat([[0,1,0,1]], repeats=num_discs, axis=0)

scat = ax.scatter(discs['position'][:, 0], discs['position'][:, 1],
                  s=disc_size, lw=0.5, edgecolors='k',
                  facecolors=discs['color'])
# disc 0
# discs['position'][0] = np.array([.5, 0.5])
discs['velocity'][0] = np.array([5*vmax, 5*vmax*1.1])
discs['color'][0] = [1,0,0,1]

dt = 0
def animate(frame, dt=dt):
    dt += 1
    x, v = move(discs['position'], discs['velocity'], dt, box_size=1)
    discs['position'], discs['velocity'] = x, v
    # 
    for idx in range(1, num_discs):
        if distance(discs['position'][0], discs['position'][idx]) < 0.0002:
            discs['color'][idx] = [1,0,0,1]
    # Update the scatter collection, with the new colors, sizes and positions.
    scat.set_facecolors(discs['color'])
    scat.set_offsets(discs['position'])
    return dt

num_loops = 1
animt = FuncAnimation(fig, animate, frames=100*num_loops, interval=30, repeat=True)
video = animt.to_html5_video()
html  = display.HTML(video)
display.display(html)
plt.close()
runtime(t0)

5 sec.


In [190]:
def distance(x1, x2):
    return np.dot(x1-x2, x1-x2)

In [192]:
distance(discs['position'][0], discs['position'][1])

0.61167878193704

In [9]:
"""
Animation of Elastic collisions with Gravity

author: Jake Vanderplas
email: vanderplas@astro.washington.edu
website: http://jakevdp.github.com
license: BSD
Please feel free to use and modify this, but keep the above information. Thanks!
"""

from scipy.spatial.distance import pdist, squareform
import scipy.integrate as integrate

t0 = time()

class ParticleBox:
    """Orbits class
    
    init_state is an [N x 4] array, where N is the number of particles:
       [[x1, y1, vx1, vy1],
        [x2, y2, vx2, vy2],
        ...               ]

    bounds is the size of the box: [xmin, xmax, ymin, ymax]
    """
    def __init__(self,
                 init_state = [[1, 0, 0, -1],
                               [-0.5, 0.5, 0.5, 0.5],
                               [-0.5, -0.5, -0.5, 0.5]],
                 bounds = [-2, 2, -2, 2],
                 size = 0.04,
                 M = 0.05,
                 G = 0):
        self.init_state = np.asarray(init_state, dtype=float)
        self.M = M * np.ones(self.init_state.shape[0])
        self.size = size
        self.state = self.init_state.copy()
        self.time_elapsed = 0
        self.bounds = bounds
        self.G = G

    def step(self, dt):
        """step once by dt seconds"""
        self.time_elapsed += dt
        
        # update positions
        self.state[:, :2] += dt * self.state[:, 2:]

        # find pairs of particles undergoing a collision
        D = squareform(pdist(self.state[:, :2]))
        ind1, ind2 = np.where(D < 2 * self.size)
        unique = (ind1 < ind2)
        ind1 = ind1[unique]
        ind2 = ind2[unique]

        # update velocities of colliding pairs
        for i1, i2 in zip(ind1, ind2):
            # mass
            m1 = self.M[i1]
            m2 = self.M[i2]

            # location vector
            r1 = self.state[i1, :2]
            r2 = self.state[i2, :2]

            # velocity vector
            v1 = self.state[i1, 2:]
            v2 = self.state[i2, 2:]

            # relative location & velocity vectors
            r_rel = r1 - r2
            v_rel = v1 - v2

            # momentum vector of the center of mass
            v_cm = (m1 * v1 + m2 * v2) / (m1 + m2)

            # collisions of spheres reflect v_rel over r_rel
            rr_rel = np.dot(r_rel, r_rel)
            vr_rel = np.dot(v_rel, r_rel)
            v_rel = 2 * r_rel * vr_rel / rr_rel - v_rel

            # assign new velocities
            self.state[i1, 2:] = v_cm + v_rel * m2 / (m1 + m2)
            self.state[i2, 2:] = v_cm - v_rel * m1 / (m1 + m2) 

        # check for crossing boundary
        crossed_x1 = (self.state[:, 0] < self.bounds[0] + self.size)
        crossed_x2 = (self.state[:, 0] > self.bounds[1] - self.size)
        crossed_y1 = (self.state[:, 1] < self.bounds[2] + self.size)
        crossed_y2 = (self.state[:, 1] > self.bounds[3] - self.size)

        self.state[crossed_x1, 0] = self.bounds[0] + self.size
        self.state[crossed_x2, 0] = self.bounds[1] - self.size

        self.state[crossed_y1, 1] = self.bounds[2] + self.size
        self.state[crossed_y2, 1] = self.bounds[3] - self.size

        self.state[crossed_x1 | crossed_x2, 2] *= -1
        self.state[crossed_y1 | crossed_y2, 3] *= -1

        # add gravity
        self.state[:, 3] -= self.M * self.G * dt


#------------------------------------------------------------
# set up initial state
np.random.seed(0)
init_state = 0 + np.random.random((10, 4))
init_state[:, :2] *= 3.9

box = ParticleBox(init_state, size=0.04)
dt = 1. / 50 # fps


#------------------------------------------------------------
# set up figure and animation
fig = plt.figure()
fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
ax = fig.add_subplot(111, aspect='equal', autoscale_on=False,
                     xlim=(-3.2, 3.2), ylim=(-2.4, 2.4))

# particles holds the locations of the particles
particles, = ax.plot([], [], 'bo', ms=6)

# rect is the box edge
rect = plt.Rectangle(box.bounds[::2],
                     box.bounds[1] - box.bounds[0],
                     box.bounds[3] - box.bounds[2],
                     ec='none', lw=2, fc='none')
ax.add_patch(rect)

def init():
    """initialize animation"""
    global box, rect
    particles.set_data([], [])
    rect.set_edgecolor('none')
    return particles, rect

def animate(i):
    """perform animation step"""
    global box, rect, dt, ax, fig
    box.step(dt)

    ms = int(fig.dpi * 2 * box.size * fig.get_figwidth()
             / np.diff(ax.get_xbound())[0])
    
    # update pieces of the animation
    rect.set_edgecolor('k')
    particles.set_data(box.state[:, 0], box.state[:, 1])
    particles.set_markersize(ms)
    return particles, rect

animt = FuncAnimation(fig, animate, frames=600,
                      interval=10, blit=True, init_func=init)

# animt = FuncAnimation(fig, animate, init_func=init, frames=100*num_loops, interval=30, repeat=True)
video = animt.to_html5_video()
html = display.HTML(video)
display.display(html)
plt.close()
runtime(t0)

22 sec.


In [6]:
animt.save('animt2.gif', writer='imagemagick')